# VinBigData Chest X-ray Abnormalities Detection usig YOLO5

![](https://www.verywellhealth.com/thmb/eUG48c__vQ0Se3So4-msAHwoqB8=/1500x0/filters:no_upscale():max_bytes(150000):strip_icc():format(webp)/iStock_22401848_MEDIUM-58262cb63df78c6f6adebb27.jpg)

Helpful kernel:
* [https://www.kaggle.com/sreevishnudamodaran/vinbigdata-fusing-bboxes-coco-dataset](http://)
* [https://www.kaggle.com/raddar/vinbigdata-competition-jpg-data-3x-downsampled](http://)

The Github repo 
https://github.com/ZFTurbo/Weighted-Boxes-Fusion[](http://)

In [ ]:
!pip install ensemble-boxes

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pydicom
import cv2
import os
from pydicom.pixel_data_handlers.util import apply_voi_lut
import matplotlib.pyplot as plt


In [ ]:
train_img_list = os.listdir('/kaggle/input/vinbigdata-chest-xray-abnormalities-detection/train/')
test_img_list  = os.listdir('/kaggle/input/vinbigdata-chest-xray-abnormalities-detection/test/')

test_dir='/kaggle/input/vinbigdata-chest-xray-abnormalities-detection/test/'
train_dir='/kaggle/input/vinbigdata-chest-xray-abnormalities-detection/train/'
train_df=pd.read_csv('/kaggle/input/vinbigdata-chest-xray-abnormalities-detection/train.csv')
sample_submission_df=pd.read_csv('/kaggle/input/vinbigdata-chest-xray-abnormalities-detection/sample_submission.csv')



In [ ]:
print("number of images in the trainig dataset = ",len(set(train_df['image_id'])))
images_ids=list(set(train_df['image_id']))


In [ ]:
print("number of images in the test dataset = ",len(set(sample_submission_df['image_id'])))
test_images_ids=list(set(sample_submission_df['image_id']))


In [ ]:
train_annotated_df=train_df[(train_df.class_name !='No finding')]
images_id_with_annotations=set(train_annotated_df['image_id'])
print("The number of images with annotations = ",len(images_id_with_annotations))
print("The number of annotations in the training dataset = ",len(train_annotated_df))
print("the average annotations per image = ",len(train_annotated_df)/len(images_id_with_annotations))

In [ ]:

image_id_to_annotation_num_dict={}
for i,image_id in enumerate(set(train_annotated_df['image_id'])):
    annotation_num=len(train_annotated_df[(train_df.image_id==image_id)])
    image_id_to_annotation_num_dict[image_id] = annotation_num


In [ ]:

import seaborn as sns

lens = list(image_id_to_annotation_num_dict.values())

plt.figure(figsize=(15,6))
sns.countplot(lens);
plt.xlabel('Number of Bounding Boxes in Images');

In [ ]:
import cv2


labels =  [
            "Aortic_enlargement",
            "Atelectasis",
            "Calcification",
            "Cardiomegaly",
            "Consolidation",
            "ILD",
            "Infiltration",
            "Lung_Opacity",
            "Nodule/Mass",
            "Other_lesion",
            "Pleural_effusion",
            "Pleural_thickening",
            "Pneumothorax",
            "Pulmonary_fibrosis"
            ]

label2color = [[59 ,238,119], [222,21 ,229], [94 , 49,164], 
               [206,221,133], [117, 75, 3 ], [210,224,119],
               [211,176,166], [63 , 7 ,197], [102, 65, 77],
               [194,134,175], [209,219,50 ], [255, 44, 47], 
               [89 ,125,149], [110,27 ,100]]
thickness=5




In [ ]:

from ensemble_boxes import *
from collections import Counter

iou_thr = 0.5
skip_box_thr = 0.0001
sigma = 0.1
###############################################################################################
'''
Function name :read_xray

Inputs:
       path:path to the dicom file(xray file)
       voi_lut ,
       fix_monochrome,
       downscale_factor : the factor to downscale the xray image 

Output: numpy array of the xray image after preprocessing and downscaling

Responsibility:
     -reads dicom file (xray file) 
     -transforms it to numpy array image 
     -make some preprocessing
     -downscale the image
'''
def read_xray(path, voi_lut = True, fix_monochrome = True, downscale_factor = 3):
    dicom = pydicom.read_file(path)

    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array

    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data

    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255.0).astype(np.uint8)
    new_shape = tuple([int(x / downscale_factor) for x in data.shape])
    data = cv2.resize(data, (new_shape[1], new_shape[0]))

    return data
###############################################################################################
'''
Function name :draw_bbox

Inputs:
       img: image array 
       bbox: [xmin,ymin,xmax,ymax] ,
       color: [r,g,b] for the bounding box color,
       class_name : the class name of the bounding box 

Output: the image after drawing the bounding box

Responsibility: draw bounding box 
     
'''

def draw_bbox(img,bbox,color,class_name):
    #print(bbox)
    cv2.rectangle(img,(int(bbox[0]), int(bbox[1])), (int(bbox[2]),int(bbox[3])),color, thickness)
    cv2.putText(img, class_name, (int(bbox[0]),int(bbox[1])-10),cv2.FONT_HERSHEY_SIMPLEX, 1.5, color,thickness=3)
    
    return img
    
###############################################################################################
'''
Function name :bounding_box_filtering

Inputs:
       train_annotated_df: the dataframe of the annotated images  
       image_basename:  ,
       filtering_function: the filtering function options: 
                                     Non-maximum Suppression (NMS)
                                     Soft-NMS
                                     Non-maximum weighted (NMW) 
                                     Weighted boxes fusion (WBF)
       
       downscale_factor : the downscale factor of image

Output: img_before : image with bounding boxes before filtering  ,
        img_afterimage :image with bounding boxes after filtering
        
Responsibility: Bounding box filtering
     
'''
def bounding_box_filtering(train_annotated_df,image_basename,filtering_function,downscale_factor):

    img_path=train_dir+image_basename+'.dicom'
    img=read_xray(img_path, voi_lut = True, fix_monochrome = True, downscale_factor = downscale_factor)
    img=cv2.cvtColor(img,cv2.COLOR_GRAY2RGB)
    img_annotations = train_annotated_df[train_annotated_df.image_id==image_basename]

    boxes_viz = (img_annotations[['x_min', 'y_min', 'x_max', 'y_max']].to_numpy()/(1.0*downscale_factor)).tolist()
    labels_viz = img_annotations['class_id'].to_numpy().tolist()
    
    
    ## Visualize Original Bboxes
    img_before = img.copy()
    for box, label in zip(boxes_viz, labels_viz):
        color = tuple(label2color[int(label)])
        img_before = draw_bbox(img_before, box,color,labels[int(label)])
    
    boxes_list = []
    scores_list = []
    labels_list = []
    weights = []
    
    boxes_single = []
    labels_single = []
    
    cls_ids = img_annotations['class_id'].unique().tolist()
    count_dict = Counter(img_annotations['class_id'].tolist())
    #print(count_dict)

    for cid in cls_ids:       
        ## Performing Fusing operation only for multiple bboxes with the same label
        if count_dict[cid]==1:
            labels_single.append(cid)
            single_box=img_annotations[img_annotations.class_id==cid][['x_min', 'y_min', 'x_max', 'y_max']].to_numpy()/(1.0*downscale_factor)
            single_box=single_box.squeeze().tolist()
            boxes_single.append(single_box)

        else:
            cls_list =img_annotations[img_annotations.class_id==cid]['class_id'].tolist()
            labels_list.append(cls_list)
            bbox = img_annotations[img_annotations.class_id==cid][['x_min', 'y_min', 'x_max', 'y_max']].to_numpy()/(1.0*downscale_factor)
            ## Normalizing Bbox by Image Width and Height
            bbox = bbox/(img.shape[1], img.shape[0], img.shape[1], img.shape[0])
            bbox = np.clip(bbox, 0, 1)
            boxes_list.append(bbox.tolist())
            scores_list.append(np.ones(len(cls_list)).tolist())

            weights.append(1)
    #print(np.array(boxes_list).squeeze().shape)
    #print(labels_list)
    # Perform NMS
    boxes, scores, box_labels = filtering_function(boxes_list, scores_list, labels_list, weights=weights,
                                    iou_thr=iou_thr)
    #print(box_labels)

    boxes = boxes*(img.shape[1], img.shape[0], img.shape[1], img.shape[0])
    boxes = boxes.round(1).tolist()
    box_labels = box_labels.astype(int).tolist()

    boxes.extend(boxes_single)
    box_labels.extend(labels_single)
        
    ## Visualize Bboxes after operation
    img_after = img.copy()
    for box, label in zip(boxes, box_labels):
        color = label2color[int(label)]
        img_after = draw_bbox(img_after,box,color,labels[int(label)])
    
    return img_before,img_after 

###############################################################################################
'''
Function name :read_image_bounding_boxes

Inputs:
       image_path: the bath to the jpg image  
       bounding_box_path: the path to the bounding box .txt in yolo format ,
       

Output: img_array_before : the image before adding bounding boxes ,
        img_array_after : the image after adding bounding boxes,
        bboxs : bounding boxes ,
        class_ids: the class id of the bounding boxes

        
Responsibility: read image and bounding boxes
     
'''


def read_image_bounding_boxes(image_path,bounding_box_path):
    img_array  = cv2.imread(image_path)
    file1 = open(bounding_box_path,"r+")
    Lines = file1.readlines()
    img_array_before=img_array.copy()

    count = 0
    # Strips the newline character
    class_ids=[]
    bboxs=[]
    for line in Lines:
        count += 1
        #print("Line{}: {}".format(count, line.strip()))
        bbox=[]
        for idx,dim in enumerate(line.strip().split()):
            if idx==0:
               class_ids.append(dim) 
            else:
               bbox.append(float(dim))
        bboxs.append(bbox)
    bboxs=np.array(bboxs) 

    bboxs=bboxs*np.array((img_array.shape[1], img_array.shape[0], img_array.shape[1], img_array.shape[0]))
    xmin=bboxs[:,0]-(bboxs[:,2])/2
    xmax=bboxs[:,0]+(bboxs[:,2])/2
    ymin=bboxs[:,1]-(bboxs[:,3])/2
    ymax=bboxs[:,1]+(bboxs[:,3])/2
    bboxs=np.stack((xmin,ymin,xmax,ymax),axis=0).T
    
    img_array_after=img_array.copy()

    for label,box in zip(class_ids,bboxs.tolist()):
        box=(np.array(box)).tolist()
        color = label2color[int(label)]
        img_array = draw_bbox(img_array_after,box,color,labels[int(label)])
    
    return img_array_before,img_array_after,bboxs,class_ids


###############################################################################################
'''
Function name :draw_bounding_box_of_image_using_image_id

Inputs:
       train_annotated_df : the dataframe of the annotated images ,
       image_id :the image id of the image,
       base_dir: the directory of the image folder,
       downscale_factor:
       

Output: None
        
Responsibility: ploting image before and after adding bounding boxes
     
'''

def draw_bounding_box_of_image_using_image_id(train_annotated_df,image_id,base_dir,downscale_factor):
    img_path=base_dir+image_id+'.dicom'
    img=read_xray(img_path, voi_lut = True, fix_monochrome = True, downscale_factor = downscale_factor)
    img=cv2.cvtColor(img,cv2.COLOR_GRAY2RGB)
    img_=img.copy()
    df=train_annotated_df[(train_annotated_df.image_id==image_id)]

    for i in range(len(df)) :

        x_min = df.iloc[i].x_min/(1.0*downscale_factor)
        y_min = df.iloc[i].y_min/(1.0*downscale_factor)
        x_max = df.iloc[i].x_max/(1.0*downscale_factor)
        y_max = df.iloc[i].y_max/(1.0*downscale_factor)
        bbox=[x_min,y_min,x_max,y_max]
        color=tuple(label2color[df.iloc[i].class_id])
        class_name=df.iloc[i].class_name.upper()
        draw_bbox(img_,bbox,color,class_name)
    fig, axes = plt.subplots(1, 2, figsize=(20,20))
    axes[0].imshow(img)    
    axes[1].imshow(img_)
    return img,img_

In [ ]:
###############################################################################################
'''
Function name : bbox_filtering

Inputs:
       img : the image array ,
       img_annotations : the data frame of the image which contians the bounding boxes info ,
       filtering_function: the filtering function that filter bounding boxes
                                options: 
                                     Non-maximum Suppression (NMS)
                                     Soft-NMS
                                     Non-maximum weighted (NMW) 
                                     Weighted boxes fusion (WBF) ,      
       downscale_factor 
       

Output: boxes : bounding boxes,
        box_labels : class_id of the bounding boxes
        
Responsibility: bounding box filtering
     
'''
def bbox_filtering(img,img_annotations,filtering_function,downscale_factor):
    boxes_list = []
    scores_list = []
    labels_list = []
    weights = []
    
    boxes_single = []
    labels_single = []
    
    cls_ids = img_annotations['class_id'].unique().tolist()
    count_dict = Counter(img_annotations['class_id'].tolist())

    for cid in cls_ids:       
        ## Performing Fusing operation only for multiple bboxes with the same label
        if count_dict[cid]==1:
            labels_single.append(cid)
            
            single_box=img_annotations[img_annotations.class_id==cid][['x_min', 'y_min', 'x_max', 'y_max']].to_numpy()/(1.0*downscale_factor)#.squeeze()
            single_box=single_box.squeeze()
            single_box = single_box/(img.shape[1], img.shape[0], img.shape[1], img.shape[0])
            single_box = np.clip(single_box, 0, 1).tolist()
            boxes_single.append(single_box)

        else:
            cls_list =img_annotations[img_annotations.class_id==cid]['class_id'].tolist()
            labels_list.append(cls_list)
            bbox = img_annotations[img_annotations.class_id==cid][['x_min', 'y_min', 'x_max', 'y_max']].to_numpy()/(1.0*downscale_factor)
            ## Normalizing Bbox by Image Width and Height
            bbox = bbox/(img.shape[1], img.shape[0], img.shape[1], img.shape[0])
            bbox = np.clip(bbox, 0, 1)
            boxes_list.append(bbox.tolist())
            scores_list.append(np.ones(len(cls_list)).tolist())

            weights.append(1)
    # Perform NMS
    #print(boxes_list)
    if boxes_list != []:
       boxes, scores, box_labels = filtering_function(boxes_list, scores_list, labels_list, weights=weights,
                                    iou_thr=iou_thr)
    #print(boxes)
    if boxes_single != [] and boxes_list != []:
       boxes=np.concatenate((boxes,np.array(boxes_single)),axis=0)
       box_labels=np.concatenate((box_labels,np.array(labels_single)),axis=0) 
    elif boxes_single != [] and boxes_list ==[]:
         boxes=np.array(boxes_single)
         box_labels=np.array(labels_single)   
    
    

    box_labels = box_labels.astype(int).tolist()
    
    return boxes,box_labels

###############################################################################################
'''
Function name : saving_image_and_filtered_bounding_boxes_in_yolo_format

Inputs:
       train_annotated_df : the dataframe of annotated images,
       image_basename : the image id , 
       downscale_factor : ,
       filtering_function :,
       path :    
       

Output: boxes : ,
        box_labels : 
        
Responsibility: save the image and the bounding boxes in yolo format
     
'''

def saving_image_and_filtered_bounding_boxes_in_yolo_format(train_annotated_df,image_basename,downscale_factor,filtering_function,path):
    path_list=path.split('/')
    #image_basename=images_id_with_annotations[idx]
    img_path=train_dir+image_basename+'.dicom'
    img=read_xray(img_path, voi_lut = True, fix_monochrome = True, downscale_factor = downscale_factor)
    img=cv2.cvtColor(img,cv2.COLOR_GRAY2RGB)
    
    cv2.imwrite(path_list[0]+'/images/'+path_list[2]+'/'+image_basename+'.jpg', img)
    
    img_annotations = train_annotated_df[train_annotated_df.image_id==image_basename]
    
    
    
    boxes,box_labels=bbox_filtering(img,img_annotations,filtering_function,downscale_factor) 
    
    x=boxes[:,0]+(boxes[:,2]-boxes[:,0])/2
    y=boxes[:,1]+(boxes[:,3]-boxes[:,1])/2
    w=boxes[:,2]-boxes[:,0]
    h=boxes[:,3]-boxes[:,1]
    boxes=np.stack((x, y,w,h), axis=1)
    
    detected_objects=np.concatenate((np.expand_dims(box_labels, axis=1),boxes), axis=1)
    
    label_file = open(path_list[0]+'/labels/'+path_list[2]+'/'+image_basename+'.txt',"w+")
    for detected_object in detected_objects.tolist() :
        label_file.write(str(int(detected_object[0]))+' '+str(detected_object[1])+' '+str(detected_object[2])+' '+str(detected_object[3])+' '+str(detected_object[4]))
        label_file.write('\n')
    label_file.close()
    
    return boxes,box_labels

###############################################################################################
'''
Function name : read_image

Inputs:
       directory: the path of the dicom (xray image) ,
       image_id: the image id ,
       downscale_factor:   
       

Output: image : the image in numpy array 
        
Responsibility: read image in dicom and transforms it to numpy array
     
'''

def read_image(directory,image_id,downscale_factor):
    path=directory+image_id+'.dicom'
    image=read_xray(path, voi_lut = True, fix_monochrome = True, downscale_factor = downscale_factor)
    image=cv2.cvtColor(image,cv2.COLOR_GRAY2RGB)
    return image
###############################################################################################
'''
Function name : rescale_bounding_boxes

Inputs:
       image : the origonal image,
       image_scaled : the scaled image,
       bboxes:  bounding boxes 
       

Output: image : the image in numpy array after adding bounding boxes after rescaling the bounding boxes
        
Responsibility: read image in dicom and transforms it to numpy array
     
'''

def rescale_bounding_boxes(image,image_scaled,bboxes):
    bboxes=bboxes*round(image.shape[0]/image_scaled.shape[0])

    for label,box in zip(class_ids,bboxes.tolist()):
        box=(np.array(box)).tolist()
        color = label2color[int(label)]
        image = draw_bbox(image,box,color,labels[int(label)])
    return image 

'''
Function name : Reformat_the_prediction

Inputs:
       test_images_path : the path to the test images ,
       labels_path : the path to the test labels,
       image_id : the image id ,
       visualize : True or false  
       

Output: image : the bounding boxes after reformating [class_id,confidence_score,xmin,ymin,xmax,ymax]
        
Responsibility: Reformating and visualizing the bounding boxes
     
'''


def Reformat_the_prediction(test_images_path,labels_path,image_id,visualize=False):
    downscale_factor=1
    image=read_image(test_dir,image_id,downscale_factor)

    image_labels=[]
    with open(test_labels_dir+image_id+'.txt') as fp:

    #with open(test_labels_dir+test_list[100]) as fp:
       line = fp.readline()
       cnt = 1
       while line:
           labels_=[] 
           for i,item in enumerate(line.strip().split()) :
               if i==0:
                  labels_.append(int(item))
               else:
                  labels_.append(float(item))
           image_labels.append(labels_)     
           line = fp.readline()
           cnt += 1    

    image_labels=np.array(image_labels)
    bboxs= image_labels[:,1:-1]   
    bboxs=np.array(bboxs) 

    bboxs=bboxs*np.array((image.shape[1], image.shape[0], image.shape[1], image.shape[0]))
    xmin=bboxs[:,0]-(bboxs[:,2])/2
    xmax=bboxs[:,0]+(bboxs[:,2])/2
    ymin=bboxs[:,1]-(bboxs[:,3])/2
    ymax=bboxs[:,1]+(bboxs[:,3])/2
    bboxs=np.stack((xmin,ymin,xmax,ymax),axis=0).T
    
    if visualize==True: 
    
        img_array_after=copy.deepcopy(image)
        class_ids=image_labels[:,0]
        for label,box in zip(class_ids.tolist(),bboxs.tolist()):
            box=(np.array(box)).tolist()
            color = label2color[int(label)]
            img_array = draw_bbox(img_array_after,box,color,labels[int(label)])
        fig, axes = plt.subplots(1, 2, figsize=(20,20))
        axes[0].imshow(image)    
        axes[1].imshow(img_array_after)        
    #else:
    return np.stack((image_labels[:,0],image_labels[:,-1],xmin,ymin,xmax,ymax),axis=0).T 

In [ ]:
def Reformat_the_prediction_(test_images_path,labels_path,image_id,test_images_shapes,index,visualize=False):
    if visualize==True:
        downscale_factor=1
        image=read_image(test_dir,image_id,downscale_factor)
    else:
        image=test_images_shapes[index]
        
    image_labels=[]
    with open(test_labels_dir+image_id+'.txt') as fp:

    #with open(test_labels_dir+test_list[100]) as fp:
       line = fp.readline()
       cnt = 1
       while line:
           labels_=[] 
           for i,item in enumerate(line.strip().split()) :
               if i==0:
                  labels_.append(int(item))
               else:
                  labels_.append(float(item))
           image_labels.append(labels_)     
           line = fp.readline()
           cnt += 1    

    image_labels=np.array(image_labels)
    bboxs= image_labels[:,1:-1]   
    bboxs=np.array(bboxs) 
    if visualize==True:
        bboxs=bboxs*np.array((image.shape[1], image.shape[0], image.shape[1], image.shape[0]))
    else:
        bboxs=bboxs*np.array((image[1], image[0], image[1], image[0]))
        
    xmin=bboxs[:,0]-(bboxs[:,2])/2
    xmax=bboxs[:,0]+(bboxs[:,2])/2
    ymin=bboxs[:,1]-(bboxs[:,3])/2
    ymax=bboxs[:,1]+(bboxs[:,3])/2
    bboxs=np.stack((xmin,ymin,xmax,ymax),axis=0).T
    
    if visualize==True: 
    
        img_array_after=copy.deepcopy(image)
        class_ids=image_labels[:,0]
        for label,box in zip(class_ids.tolist(),bboxs.tolist()):
            box=(np.array(box)).tolist()
            color = label2color[int(label)]
            img_array = draw_bbox(img_array_after,box,color,labels[int(label)])
        fig, axes = plt.subplots(1, 2, figsize=(20,20))
        axes[0].imshow(image)    
        axes[1].imshow(img_array_after)        
    return np.stack((image_labels[:,0],image_labels[:,-1],xmin,ymin,xmax,ymax),axis=0).T 

def no_filtering(boxes_list, scores_list, labels_list, weights,iou_thr):
    boxes=[]
    labels_=[]
    #print(labels_list)
    for boxes_list_cls,labels_cls in zip(boxes_list,labels_list):
        boxes+=boxes_list_cls
        labels_+=labels_cls
    boxes=np.array(boxes)
    labels_cls=np.array(labels_)
    return boxes,np.array(scores_list),labels_cls

# Spliting the data to train and validation sets

In [ ]:
from sklearn.model_selection import train_test_split
train_list,val_list=train_test_split(list(images_id_with_annotations), test_size=0.2, random_state=42)

# The visualization of the training images and their Bounding Boxes

In [ ]:
for i in range(6):
   draw_bounding_box_of_image_using_image_id(train_annotated_df,train_list[i],train_dir,1)  

# The visualization of the training data and the filtering of the bounding boxes:
The repo for bounding boxes filtering : [https://github.com/ZFTurbo/Weighted-Boxes-Fusion](http://)
## Filter function options:
* no_filtering
* nms
* soft_nms
* non_maximum_weighted
* weighted_boxes_fusion

## in this competition the filtering did not help in the LP score in my experience 

In [ ]:
img_before,img_after=bounding_box_filtering(train_annotated_df,train_list[10],no_filtering,2)
fig, axes = plt.subplots(1, 2, figsize=(20,20))
axes[0].imshow(img_before)    
axes[1].imshow(img_after)


In [ ]:
img,img_=draw_bounding_box_of_image_using_image_id(train_annotated_df,train_list[5],train_dir,1)

# Creating the directories for training ,validation and test 

In [ ]:
import os

# define the name of the directory to be created
path = "vinbigdata/labels/train/"

try:
    os.makedirs(path)
except OSError:
    print ("Creation of the directory %s failed" % path)
else:
    print ("Successfully created the directory %s" % path)
###############################################################
path = "vinbigdata/labels/val/"

try:
    os.makedirs(path)
except OSError:
    print ("Creation of the directory %s failed" % path)
else:
    print ("Successfully created the directory %s" % path)    
####################################################################################################    
path = "vinbigdata/images/train/"

try:
    os.makedirs(path)
except OSError:
    print ("Creation of the directory %s failed" % path)
else:
    print ("Successfully created the directory %s" % path)
##############################################################
path = "vinbigdata/images/val/"

try:
    os.makedirs(path)
except OSError:
    print ("Creation of the directory %s failed" % path)
else:
    print ("Successfully created the directory %s" % path)    
        
path = "vinbigdata/test/"

try:
    os.makedirs(path)
except OSError:
    print ("Creation of the directory %s failed" % path)
else:
    print ("Successfully created the directory %s" % path)     

In [ ]:
path='vinbigdata/images/train/'
boxes,labels_=saving_image_and_filtered_bounding_boxes_in_yolo_format(train_annotated_df,train_list[10],3,no_filtering,path)


In [ ]:
print("The number of images in the training set : ",len(train_list))
print("The number of images in the vaildation set : ",len(val_list))
print("The number of images in the test set : ",len(test_images_ids))


# Creating the training data , validation data and test data folders for the YOLO5 :
The details about the directory structure is mentioned in this link : [https://github.com/ultralytics/yolov5/wiki/Train-Custom-Data](http://)


remove [0:10] from train_list,val_list,test_list to use all the data available 

In [ ]:

downscale_factor=3

path='vinbigdata/images/train/'
for idx,image_basename in enumerate(train_list[0:10]):
    print("train image : ",idx)
    boxes,labels_=saving_image_and_filtered_bounding_boxes_in_yolo_format(train_annotated_df,image_basename,downscale_factor,no_filtering,path)

print("************************************************************************")    
path='vinbigdata/images/val/'
for idx,image_basename in enumerate(val_list[0:10]):
    print("val image : ",idx)
    boxes,labels_=saving_image_and_filtered_bounding_boxes_in_yolo_format(train_annotated_df,image_basename,downscale_factor,no_filtering,path)
   
print("************************************************************************")    
test_path = "vinbigdata/test/"
for idx,image_basename in enumerate(test_images_ids[0:10]):
    print("test image : ",idx) 
    image=read_image(test_dir,image_basename,downscale_factor)
    save_path=test_path+str(image_basename)+'.jpg'
    cv2.imwrite(save_path,image)
    

In [ ]:
  

image_path='vinbigdata/images/train/'+train_list[0]+'.jpg'
bounding_box_path='vinbigdata/labels/train/'+train_list[0]+'.txt'
image_scaled,image_bbox_scaled,bboxes,class_ids=read_image_bounding_boxes(image_path,bounding_box_path)
img_array=read_image(train_dir,train_list[0],1)
img_array=rescale_bounding_boxes(img_array,image_scaled,bboxes)

fig, axes = plt.subplots(1, 3, figsize=(20,20))
axes[0].imshow(image_scaled)    
axes[1].imshow(image_bbox_scaled)
axes[2].imshow(img_array)

 

# YOLO5
The repo for YOLO5 pytorch [https://github.com/ultralytics/yolov5](http://) 

Train on custom data: [https://github.com/ultralytics/yolov5/wiki/Train-Custom-Data](http://)


In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt  # install dependencies

import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

# Training using yolov5 
*  ### change the number of epoches to bigger number
* ### in --weights use yolov5x.pt instead of yolov5x.pt

In [ ]:
# Train YOLOv5s on COCO128 for 3 epochs
!WANDB_MODE="dryrun" python train.py --img 640 --batch 16 --epochs 1 --data /kaggle/input/yaml-yolo5-vinbigdata/VinBigData.yaml --weights yolov5s.pt --cache 

# Inference on testset 


In [ ]:
#!WANDB_MODE="dryrun" python test.py --weights /kaggle/working/yolov5/runs/train/exp/weights/last.pt --data coco128.yaml --conf-thres 0.7 --task test --save-conf --save-txt 

In [ ]:
'''
!zip -r vinbigdata_00.zip vinbigdata
from IPython.display import FileLink
FileLink(r'vinbigdata_00.zip')
'''

# Making the submission file
## uncomment next lines to make the submission file

In [ ]:
'''
import os
test_labels_dir='/kaggle/working/yolov5/runs/test/exp/labels/'
test_list=os.listdir(test_labels_dir)
len(test_list)
test_images_shapes=np.load('../input/labes-images-shapes/content/labels_images_shapes/test_images_shapes.npy').tolist()
'''

In [ ]:
'''
test_path = "/kaggle/working/vinbigdata/test/"

prediction_strings_list=[]
for i,image_id in enumerate(sample_submission_df['image_id'].tolist()):
    if image_id+'.txt' in test_list:  
        #predictions=Reformat_the_prediction(test_path,test_labels_dir,image_id,False)
        predictions=Reformat_the_prediction_(test_path,test_labels_dir,image_id,test_images_shapes,i,False)
        prediction_string=''
        print(i," of ",len(sample_submission_df))
        for preds in predictions.tolist():
            for pred in preds:
                #print(pred)
                prediction_string+=str(pred)+' '
    else:
        prediction_string='14 1.0 0 0 1 1 '
    prediction_strings_list.append(prediction_string[:-1])
'''    

In [ ]:
'''
sample_submission_df['image_id'].tolist()
sample_submission_df['PredictionString']=np.array(prediction_strings_list)
sample_submission_df.to_csv("submission_013.csv",index=False)
''''

In [ ]:
'''
from IPython.display import FileLink
FileLink(r'submission_013.csv')
'''

In [ ]:
#ls vinbigdata/images/train/
#len(os.listdir('vinbigdata/test/'))
#!unzip vinbigdata.zip
#os.stat('vinbigdata.zip').st_size/(1024*1024*1024)
#ls "vinbigdata/test/"